In [1]:
def _MA_screen(symbol, df, x):
    df['SMA'] = round(df['Adj Close'].rolling(window=x).mean(), 2)  # Simple Moving Average
    # TEMA triple exponential moving average | EMA Exponential Moving Average
    df['EMA1'] = round(df['Adj Close'].ewm(span=x, adjust=False).mean(), 2)
    df['EMA2'] = round(df['EMA1'].ewm(span=x, adjust=False).mean(), 2)
    df['EMA3'] = round(df['EMA2'].ewm(span=x, adjust=False).mean(), 2)
    df['TEMA'] = 3 * (df['EMA1'] - df['EMA2']) + df['EMA3']

    # up/down from previous 2 days
    df['SMA_2'] = df.SMA.shift(2)
    df['TEMA_2'] = df.TEMA.shift(2)
    df['SMA_up'] = df.apply(lambda x: '+' if x['SMA'] >= x['SMA_2'] else '-', axis=1)
    df['TEMA_up'] = df.apply(lambda x: '+' if x['TEMA'] >= x['TEMA_2'] else '-', axis=1)

    # reverse up/down
    df['SMA_up_1'] = df.SMA_up.shift(1)
    df['TEMA_up_1'] = df.TEMA_up.shift(1)
    df = df.dropna(subset=['SMA', 'SMA_2', 'TEMA', 'TEMA_2', 'SMA_up_1', 'TEMA_up_1'])
    df['SMA_up_rev'] = df.apply(lambda x: x['SMA_up'] if x['SMA_up'] != x['SMA_up_1'] else '', axis=1)
    df['TEMA_up_rev'] = df.apply(lambda x: x['TEMA_up'] if x['TEMA_up'] != x['TEMA_up_1'] else '', axis=1)

    SMA_seq = ''
    TEMA_seq = ''
    seq = 0
    for i in df.index[::-1]:  # look backwards
        seq += 1
        if bool(SMA_seq) & bool(TEMA_seq):
            break
        if (not SMA_seq) & bool(df['SMA_up_rev'][i]):
            SMA_seq = df['SMA_up_rev'][i] + str(seq)
        if (not TEMA_seq) & bool(df['TEMA_up_rev'][i]):
            TEMA_seq = df['TEMA_up_rev'][i] + str(seq)
    return [symbol, TEMA_seq, SMA_seq]


In [2]:
def _MA_screen_list(stockList, x, filename):
    stockList = list(set(stockList))  # make elements unique
    stockList = sorted(stockList)
    rows = []
    for symbol in stockList:
        try:
            with contextlib.redirect_stdout(io.StringIO()):
                df_yh = yf.download(symbol, period='120d')
            row = _MA_screen(symbol, df_yh, x)
            rows.append(row)
        except Exception:
            print(symbol)
    out_df = pd.DataFrame(rows, columns=['symbol', 'TEMA_seq', 'SMA_seq'])

    out_df.TEMA_seq = out_df.TEMA_seq.astype(float)
    out_df.SMA_seq = out_df.SMA_seq.astype(float)
    out_df = out_df.sort_values(by=['TEMA_seq', 'SMA_seq'], ascending=False)

    print(out_df)
    out_df.to_csv(DIR + filename, index=False)

In [3]:
def _MA_screen_list_h(stockList, x, filename):
    stockList = list(set(stockList))  # make elements unique
    stockList = sorted(stockList)
    rows = []
    for symbol in stockList:
        try:
            with contextlib.redirect_stdout(io.StringIO()):
                df_yh = yf.download(symbol, period="20d", interval="1h")
            row = _MA_screen(symbol, df_yh, x)
            rows.append(row)
        except Exception:
            print(symbol)
    out_df = pd.DataFrame(rows, columns=['symbol', 'TEMA_seq', 'SMA_seq'])

    out_df.TEMA_seq = out_df.TEMA_seq.astype(float)
    out_df.SMA_seq = out_df.SMA_seq.astype(float)
    out_df = out_df.sort_values(by=['TEMA_seq', 'SMA_seq'], ascending=False)

    print(out_df)
    out_df.to_csv(DIR + filename, index=False)

In [15]:
# C reverse up +1: buy to close C
import contextlib
import io

import pandas as pd
import yfinance as yf

yf.pdr_override()
pd.options.mode.chained_assignment = None  # default='warn'

x = 20
DIR = 'E:\Google Drive\data'

df = pd.read_csv(DIR + '\out_fidelity.csv')
df = df[(df['Option'] == 'C')]
stockList = df['Ticker'].tolist()

_MA_screen_list_h(stockList, x, '\out_Python_MA_C.csv')


   symbol  TEMA_seq  SMA_seq
9      JD       7.0     33.0
19     XL       7.0    -41.0
20   XPEV       6.0    -67.0
8     INO       2.0    -22.0
0     AMC       2.0    -27.0
10    NIO       1.0    -18.0
17   SUNW       1.0    -21.0
3    BIGC       1.0    -27.0
14   PLUG       1.0    -68.0
11   NNOX      -4.0    -18.0
5     BTX      -9.0    -76.0
15   SOXL     -12.0     -7.0
12   NVDA     -17.0    -12.0
1     AMD     -17.0    -13.0
18   TSLA     -18.0     -8.0
2    ASTS     -19.0    -12.0
7    CCIV     -20.0    -14.0
4    BLDP     -21.0    -20.0
6    BYND     -21.0    -68.0
13   PLTR     -24.0    -21.0
16   SPCE     -30.0    -27.0


In [13]:
# P C2 reverse dn -1: buy to close P or sell C
df = pd.read_csv('E:\Google Drive\data\out_fidelity.csv')
df = df[(df['Option'] == 'P') | df['C2'].notnull()]
stockList = df['Ticker'].tolist()

_MA_screen_list_h(stockList, x, '\out_Python_MA_P.csv')

   symbol  TEMA_seq  SMA_seq
33    TZA      18.0     12.0
23   NUGT      11.0     18.0
29    RKT       6.0    -25.0
18    JPM       5.0    -10.0
26     QS       5.0    -67.0
17     JD       4.0     30.0
14   DOOR       4.0    -12.0
12    DIS       3.0     32.0
28     RF       3.0    -10.0
1     AMC       3.0    -24.0
20     LC      -1.0    -13.0
6    BTWN      -1.0    -18.0
25   PLUG      -2.0    -65.0
30   ROOT      -2.0    -67.0
9      CI      -4.0     -3.0
4    ATSG      -6.0     -4.0
7     BTX      -6.0    -73.0
27   RAAS      -9.0    -58.0
16   INTC     -10.0     -4.0
22   NCTY     -10.0    -16.0
2     API     -11.0    -10.0
11   CRON     -12.0    -10.0
35   UBER     -12.0    -11.0
32   TLRY     -13.0     -9.0
19   LAZR     -14.0    -12.0
13  DISCA     -15.0    -10.0
3    ASTS     -16.0     -9.0
8     CAT     -18.0    -11.0
5    BLDP     -18.0    -17.0
34    UAL     -18.0    -18.0
21    LVS     -19.0    -12.0
15   FCEL     -19.0    -18.0
24   PLTR     -21.0    -18.0
0    ABNB     

In [70]:
def _pickUpDn(limitDay, filename):
    out_df = pd.read_csv(DIR + filename)
    today = dt.datetime.now().strftime('%d')
    upList = out_df[(0 <= out_df['TEMA_seq']) & (out_df['TEMA_seq'] <= limitDay)]  # & (out_df['SMA_seq'] < -9)
    print(upList)
    upList['symbol'].to_csv('..\data\out_PythonRevUp' + today + '.txt', index=False, header=False)

    dnList = out_df[(-limitDay <= out_df['TEMA_seq']) & (out_df['TEMA_seq'] <= 0)]  # & (out_df['SMA_seq'] > 9)
    print(dnList)
    dnList['symbol'].to_csv('..\data\out_PythonRevDn' + today + '.txt', index=False, header=False)


In [71]:
# Scan
df = pd.read_csv('..\data\in_stockList.csv', sep='[";"]', engine='python')  # 174 0:01:55 | 0:03:52
stockList = []
for i in df.index:
    symbolList = df['Symbol'][i]
    for x in symbolList.split(','):
        stockList.append(x.split(':')[-1])
        
filename = '\out_Python_MA.csv'
_MA_screen_list(stockList, x, filename)
_pickUpDn(limitDay, filename)


AAPL
ABBV
ABNB
ACB
ADBE
AFRM
AIZ
ALK
AMC
AMD
AMT
AMZN
API
ARE
ARKF
ARKG


KeyboardInterrupt: 

In [76]:
from pandas import DataFrame
from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
lst = stock_data.get_stocks_by_index('DOW JONES')  # NASDAQ 100 [98 x 10] 06:48 | S&P 500 [491 x 10] 10:34 | DOW JONES 30 04:24
df = DataFrame(lst)
stockList = df['symbol'].tolist()  # get list from a column

filename = '\out_Python_MA_SP.csv'
_MA_screen_list(stockList, x, filename)
# _pickUpDn(3, filename)

AAPL
AMGN
AXP
BA
CAT
CRM


KeyboardInterrupt: 

In [60]:
# 1 symbol
import contextlib
import io

import pandas as pd
import yfinance as yf

yf.pdr_override()
pd.options.mode.chained_assignment = None  # default='warn'

symbol = 'QS'
with contextlib.redirect_stdout(io.StringIO()):
#     df_yh = yf.download(symbol, period="120d")
    df_yh = yf.download(symbol, period="20d", interval="1h")
print(_MA_screen(symbol, df_yh, 20))


['QS', '+1', '-63']
